In [1]:
import os
import torch
import esm
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import pandas as pd

# ESM 모델과 배치 컨버터를 로드합니다.
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()

# 모델을 평가 모드로 설정하고 GPU로 이동
model.eval()
model.cuda()

# DataProvider 클래스를 임포트하고 데이터 로더를 설정합니다.
from dataprovider import DataProvider

# 데이터 경로 설정
epi_path = '../../data/deepneo/mhc1.testset.csv'
hla_path = '../../data/deepneo/HLAseq.csv'

# DataProvider 인스턴스 생성
data_provider = DataProvider(epi_path, hla_path)

# 데이터 로더 설정
data_loader = DataLoader(data_provider, batch_size=16, shuffle=False, collate_fn=lambda x: x)

# HLA 유형별 보편적인 서열
common_hla_sequence_A_or_B = "SHSMRYFYTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRMEPRAPWIEQEGPEYWDRETQIVKANSQTDRESLRTLRGYYNQSEAGSHTIQRMYGCDVGPDGRLLRGYNQYAYDGKDYIALNEDLRSWTAADTAAQITQRKWEAARVAEQLRAYLEGTCVEWLRRYLENGKETLQRA******************************************************************************************************************************************"  # HLAA 또는 HLAB 보편적인 시퀀스
common_hla_sequence_C = "MRYFYTAVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRGEPRAPWVEQEGPEYWDRETQKYKRQAQADRVSLRNLRGYYNQSEAGSHTLQRMYGCDLGPDGRLLRGYDQSAYDGKDYIALNEDLRSWTAADTAAQITQRKWEAAREAEQLRAYLEGTCVEWLRRYLENGKETLQRAEPKTHVTHHPSDHEATLRCWALGFYPT**************DTELVETRPAGDGTFQKWAAVVVPSGEQRYTCHQHEGLEPLTL*W********************************************************************"       # HLAC 보편적인 시퀀스

def clean_hla_sequence(sequence):
    """HLA 서열에서 패딩된 '*' 문자를 제거합니다."""
    return sequence.replace('*', 'X')

def embed_sequence(sequence, batch_converter, model):
    """서열을 ESM 모델로 임베딩하는 함수."""
    labels = [("sequence", sequence)]
    _, _, tokens = batch_converter(labels)
    
    with torch.no_grad():
        results = model(tokens.cuda(), repr_layers=[33], return_contacts=False)
    
    return results["representations"][33][0, 1: len(sequence) + 1].mean(0)

def encode_sequences(data_loader, batch_converter, model, common_hla_sequence_A_or_B, common_hla_sequence_C):
    sequence_representations = []
    
    # 보편적인 HLA 서열을 클린하고 임베딩 (A/B와 C 각각)
    cleaned_common_hla_A_or_B = clean_hla_sequence(common_hla_sequence_A_or_B)
    cleaned_common_hla_C = clean_hla_sequence(common_hla_sequence_C)
    
    common_hla_embedding_A_or_B = embed_sequence(cleaned_common_hla_A_or_B, batch_converter, model).cpu().numpy()
    common_hla_embedding_C = embed_sequence(cleaned_common_hla_C, batch_converter, model).cpu().numpy()
    
    for batch in tqdm(data_loader):
        hla_names, epi_seqs, targets, hla_seqs = zip(*batch)
        
        # HLA 서열에서 '*'를 제거합니다.
        cleaned_hla_seqs = [clean_hla_sequence(hla_seq) for hla_seq in hla_seqs]
        
        # HLA 시퀀스를 각각 임베딩하고 보편적인 서열과 차이를 계산
        hla_embeddings_diff = []
        for i, hla_name in enumerate(hla_names):
            try:
                if hla_name.startswith("HLAC"):
                    hla_embedding = embed_sequence(cleaned_hla_seqs[i], batch_converter, model).cpu().numpy()
                    hla_embeddings_diff.append(hla_embedding - common_hla_embedding_C)
                else:
                    hla_embedding = embed_sequence(cleaned_hla_seqs[i], batch_converter, model).cpu().numpy()
                    hla_embeddings_diff.append(hla_embedding - common_hla_embedding_A_or_B)
            except Exception as e:
                print(f"Error processing HLA sequence: {hla_name}")
                print(f"Sequence: {hla_seqs[i]}")
                print(f"Exception: {e}")
                continue  # 문제 발생 시 해당 시퀀스를 건너뜀
        
        # 길이 확인
        if len(hla_embeddings_diff) != len(batch):
            print(f"Warning: hla_embeddings_diff 길이가 {len(hla_embeddings_diff)} 이고 batch 길이는 {len(batch)} 입니다. 일치하지 않음.")
            continue  # 일치하지 않으면 이 배치는 건너뜀

        # Epitope 시퀀스를 각각 임베딩
        epitope_embeddings = [embed_sequence(epi_seq, batch_converter, model).cpu().numpy() for epi_seq in epi_seqs]
        
        # 임베딩 결과를 저장
        for i, (hla_name, epi_seq, target, hla_seq) in enumerate(batch):
            sequence_representations.append({
                'hla_name': hla_name,
                'epi_seq': epitope_embeddings[i],  # Epitope 임베딩을 저장
                'target': target,
                'hla_seq': hla_seq,  # 보편적인 시퀀스와 차이를 저장
                'hla_embedding_diff': hla_embeddings_diff[i],
            })
    
    return sequence_representations


# 인코딩된 시퀀스를 얻습니다.
encoded_sequences = encode_sequences(data_loader, batch_converter, model, common_hla_sequence_A_or_B, common_hla_sequence_C)

# 인코딩된 결과를 저장합니다.
np.save('hlaembedding.npy', [seq['hla_embedding_diff'] for seq in encoded_sequences])

# Epitope 임베딩도 저장합니다.
np.save('epitopeembedding.npy', [seq['epi_seq'] for seq in encoded_sequences])

# 추가적으로 메타데이터도 저장할 수 있습니다.
meta_data = [{
    'hla_name': seq['hla_name'],
    'epi_seq': seq['epi_seq'],  # Epitope 임베딩을 메타데이터에 저장
    'target': seq['target'],
    'hla_seq': seq['hla_seq'],
} for seq in encoded_sequences]

meta_df = pd.DataFrame(meta_data)
meta_df.to_csv('metadata.csv', index=False)


Number of HLA alleles: 7045
Number of samples: 102180


  0%|          | 8/6387 [00:03<50:40,  2.10it/s]


KeyboardInterrupt: 